In [1]:
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
import pickle
import horus as ho

In [2]:
from simsopt.configs import get_w7x_data
curves, currents, ma = get_w7x_data()

In [3]:
default_config = [curr.current for curr in currents]

In [4]:
from simsopt.field import Current, coils_via_symmetries
# GYM00+1750
currents = [Current(1.109484 * 1e6) for _ in range(5)]
currents.append(Current(-0.3661 * 1e6))
currents.append(Current(-0.3661 * 1e6))

gym_config = [curr.current for curr in currents]

# # 
# currents = [Current(1.109484) * 1e6 for _ in range(5)]
# currents.append(Current(-0.3661) * 1e6)
# currents.append(Current(-0.3661) * 1e6)

# currents = [Current(1.109484) * 1e6 for _ in range(5)]
# currents.append(Current(-0.3661) * 1e6)
# currents.append(Current(-0.3661) * 1e6)

In [5]:
print("Default config: ", default_config)
print("Gym config: ", gym_config)

Default config:  [1620000.0, 1620000.0, 1620000.0, 1620000.0, 1620000.0, 0.0, 0.0]
Gym config:  [1109484.0, 1109484.0, 1109484.0, 1109484.0, 1109484.0, -366100.0, -366100.0]


In [ ]:
coils = coils_via_symmetries(curves, currents, 5, True)

In [ ]:
from pyoculus.problems import SimsoptBfieldProblem, surf_from_coils

In [ ]:
surf = surf_from_coils(coils, ncoils=7, mpol=7, ntor=7)

In [ ]:
surf.plot(engine="mayavi", show=False, opacity=0.5)

for coil in coils:
    coil.plot(engine="mayavi", show=False)

mlab.show()

In [ ]:
R0, _, Z0 = ma.gamma()[0,:]
ps = SimsoptBfieldProblem.from_coils(R0=R0, Z0=Z0, Nfp=5, coils=coils, interpolate=True, ncoils=7, mpol=7, ntor=7, n=40)

In [ ]:
ps._mf_B.estimate_error_B(10000)

In [ ]:
ps._mf_B.estimate_error_GradAbsB(1000)

#### Plot the current difference

In [ ]:
curr_diff = []
for actual, base in zip(currents,  w7x[1]):
    print(base.current, " | ", actual.current_to_scale.current * 1e6)
    curr_diff.append(actual.current_to_scale.current * 1e6 - base.current)

In [ ]:
curr_diff

In [ ]:
cmap = plt.colormaps['RdBu']
norm = plt.Normalize(min(curr_diff), max(curr_diff))
colors = cmap(norm(curr_diff))
colors = colors[:, :3]

In [ ]:
def rep(data):
            if close:
                return np.concatenate((data, [data[0]]))
            else:
                return data

In [ ]:
# Assuming curr_diff is a list of differences in currents
# Normalize curr_diff to the range [0, 1] for coloring

# Plot the coils
for i, coil in enumerate(coils):
    c


mlab.colorbar(orientation='vertical', title='Current Difference')
mlab.show()

In [ ]:
from mayavi import mlab
for i, coil in enumerate(coils):
    coil.plot(engine="mayavi", show=False, color=tuple(colors[i%7]))

# get the current lut manager
mlab.colorbar(orientation='vertical', title='Current Difference')
mlab.show()

#### Poincare

In [ ]:
for coil in coils:
    coil.plot(engine="mayavi", show=False)

mlab.show()

In [ ]:
phis = [0]    #[(i / 4) * (2 * np.pi / nfp) for i in range(4)]

nfieldlines = 1
Rs = np.linspace(6.05, 6.2, nfieldlines)
Zs = [ma.gamma()[0, 2] for _ in range(nfieldlines)]
RZs = np.array([[r, z] for r, z in zip(Rs, Zs)])

nfieldlines = 2
p1 = np.array([5.6144507858315915, -0.8067790944375764])
p2 = np.array([5.78, -0.6])
Rs = np.linspace(p1[0], p2[0], nfieldlines)
Zs = np.linspace(p1[1], p2[1], nfieldlines)
Rs, Zs = np.meshgrid(Rs, Zs)
RZs2 = np.array([[r, z] for r, z in zip(Rs.flatten(), Zs.flatten())])

RZs = np.concatenate((RZs, RZs2))

In [ ]:
ax.plot(RZs[:, 0], RZs[:, 1], '+')
fig

In [ ]:
tys, phis = ho.poincare(bsh, RZs, phis, sc_fieldline, tol = 1e-10, plot=False)

In [ ]:
_, _, fig, ax = ho.poincare(bsh, RZs, phis, sc_fieldline, tol = 1e-10)

In [ ]:
for col in ax[0,0].collections:
    col.set_color('black')
    col.set_sizes([0.5])

In [ ]:
fig

In [ ]:
# import pickle
# pickle.dump(fig, open("../output/poincare_GYM000+1750_chaotic_black.pickle", "wb"))

#### Finding fix-point

In [ ]:
fig = pickle.load(open("../../runs/w7x-gym00-1750/W7X_GYM000-1750_poincare_black.pkl", "rb"))
ax = fig.get_axes()[0]
plt.show()

In [ ]:
# guess = [5.6, 0.8] # O point
guess = [5.7, 0.5] # X point
# guess = [5.78, -0.6]
ax.scatter(guess[0], guess[1], c='r', s=10)
fig

In [ ]:
gamma = ho.trace(ps._mf, 40*2*np.pi, [5.72, 0, 0.68], tol=1e-9)

In [ ]:
from mayavi import mlab
for coil in coils:
    coil.plot(engine="mayavi", show=False)

mlab.plot3d(gamma[0, :], gamma[1, :], gamma[2, :], tube_radius=0.01, color=(0, 0, 1))

mlab.show()

In [ ]:
from pyoculus.problems import FixedPoint

In [ ]:
# set up the integrator
iparams = dict()
iparams["rtol"] = 1e-9

pparams = dict()
pparams["nrestart"] = 0
pparams["tol"] = 1e-18
pparams['niter'] = 100

fp = FixedPoint(ps, pparams, integrator_params=iparams)

In [ ]:
fp.compute(guess=guess, pp=5, qq=4, sbegin=5.2, send=6.2, checkonly=True)

In [ ]:
results = [list(p) for p in zip(fp.x, fp.y, fp.z)]

In [ ]:
results

In [ ]:
for rr in results:
    ax.scatter(rr[0], rr[2], marker="X", edgecolors="black", linewidths=1)
fig

#### Tracing stable and unstable manifolds

In [ ]:
fig = pickle.load(open("../../runs/w7x-gym00-1750/W7X_GYM000-1750_poincare_chaotic_black.pkl", "rb"))
ax = fig.get_axes()[0]

In [ ]:
fig.set_size_inches(12, 12)
fig

In [ ]:
from pyoculus.solvers import Manifold

In [ ]:
iparam = dict()
iparam["rtol"] = 1e-12

mp = Manifold(fp, ps, integrator_params=iparam)

In [ ]:
mp.choose(0, 1, ["u+","s+"], [1, 1])

In [ ]:
mp.find_epsilon(mp.rfp_s, mp.vector_s, eps_guess = 1e-2, direction=-1)

In [ ]:
mp.compute(nintersect = 6, epsilon= 1e-3, neps = 50)

In [ ]:
mp.plot(ax=ax)

In [ ]:
fig

In [ ]:
mp.find_epsilon(mp.rfp_u, mp.vector_u)

In [ ]:
mp.error_linear_regime(1e-4, mp.rfp_u, mp.vector_u)

In [ ]:
mp.error_linear_regime(1e-4, mp.rfp_s, mp.vector_s, direction=-1)

In [ ]:
mp.integrate_single(mp.rfp_s, 1, -1, ret_jacobian=False)

In [ ]:
mp.integrate_single(mp.rfp_s, 1, -1, ret_jacobian=True)

In [ ]:
eps_s_1, eps_u_1 = mp.find_homoclinic(1e-4, 1e-4, jac=False)

In [ ]:
fund = mp.fundamental_segment
guess_i = [fund[0][1]*np.power(mp.lambda_s, 1/4), fund[1][0]*np.power(mp.lambda_u, 1/4)]
print(f"initial guess: {guess_i}")

In [ ]:
r1 = mp.integrate_single(mp.rfp_u + guess_i[1] * mp.vector_u, 5, ret_jacobian=False)
r2 = mp.integrate_single(mp.rfp_s + guess_i[0] * mp.vector_s, 5, -1, ret_jacobian=False)

In [ ]:
ax.scatter(*r1, marker="s", edgecolors="black", linewidths=1)
ax.scatter(*r2, marker="o", edgecolors="black", linewidths=1)
fig

In [ ]:
mp.error_linear_regime(guess_i[0], mp.rfp_s, mp.vector_s, direction=-1)

In [ ]:
eps_s_i, eps_u_i = mp.find_homoclinic(*guess_i, bounds = fund, n_s = 5, n_u = 5, jac=False)

In [ ]:
marker = ["P", "o"] #, "s", "p", "P", "*", "X", "D", "d", "^", "v", "<", ">"]
for i, clinic in enumerate(mp.clinics):
    eps_s_i, eps_u_i = clinic[1:3]

    n_u = 8
    # hs_i = mp.integrate(mp.rfp_s + eps_s_i * mp.vector_s, n_s, -1)
    hu_i = mp.integrate(mp.rfp_u + eps_u_i * mp.vector_u, n_u, 1)
    # ax.scatter(hs_i[0,:], hs_i[1,:], marker=marker[i], color="purple", zorder=10)
    ax.scatter(hu_i[0,:], hu_i[1,:], marker=marker[i], color="royalblue", edgecolor='cyan', zorder=10)

In [ ]:
fig

In [ ]:
mp.clinics

In [ ]:
ps.A([6.,0.,0.])

In [ ]:
mp.integrate_single(mp.rfp_u, 1, 1, ret_jacobian=False, integrate_A=True)

In [ ]:
mp.resonance_area()

In [ ]:
mp.fundamental_segment